In [1]:
import os,sys,glob
import pandas as pd
import numpy as np
import itertools

201221-15:36:00,599 nipype.utils INFO:
	 Running nipype version 1.5.0-rc1 (latest: 1.5.1)


In [2]:

logfiles_dir = '../logfiles/'
confounds_dir ='../confounds/'
design_dir = '../add_visual_reg/'


TR = .001
N_TR = 552
TR_Length = 1

subjects_str = ['04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','23','25','26','28','29'] 
video_list = list(range(1,37))

# load pre-calculated visual property for videos and other
video_visual = glob.glob('all_Visual_Property.csv')
video_visual_info = pd.read_csv(video_visual[0])
other_visual = glob.glob('RatingFixation_Visual_Property.csv')
other_visual_info = pd.read_csv(other_visual[0])



In [4]:
def video_n(video_name):
    # category
    if video_name[0:2]=='he':
        temp = 0
    elif video_name[0:2] == 'so':
        temp = 12
    else:
        temp = 24
    num = video_name[-5:-4]
    # high vs low
    if video_name[-9:-7] == 'lo':
        temp = temp + 6
        
    # social_high_4_replacement  
    if num == 't':
        temp = 16
    else:
        num = int(num)
        temp = temp + num
    return int(temp)


def get_vcat(vn):
    if vn in list(range(1,13)):
        vcat = 'heights'
    elif vn in list(range(13,25)):
        vcat = 'social'
    else: # list(range(25,37)) + list(range(61,73))
        vcat = 'spiders'
        
    return vcat  

In [6]:


for s in subjects_str:
    print(s)
    # load design matrix information
    file = glob.glob(logfiles_dir + 'AffVids_logfile_' + str(s) + '_edited.txt')
    dm_info = np.loadtxt(file[0],skiprows=1,dtype =str)
    dm_info = pd.DataFrame(dm_info)
    dm_info = dm_info.reset_index(drop=True)
    
    for r in range(1,4):
        ThreeColumns = pd.DataFrame(np.zeros((N_TR,3)),columns = ['luminance','contrast','complexity'])
        r = str(r)
        
        arr = []
        
        # add video visual property regressors
        # c_indx = ith video in the order it was presented in the experiment
        c_indx = dm_info[dm_info[1]==r].index
        for c in c_indx:
            # get the video name of the ith video
            v_name = dm_info.iloc[c,0]
            # get the video number based on the name
            v_num = video_n(v_name)
            # use the video number to retrieve the video visual property
            temp = video_visual_info[video_visual_info.video_number==v_num]
            
            print(str(c) + 'th video')
            video_onset = round(float(dm_info[dm_info[1]==r][9][c]))
            units = np.arange(video_onset, video_onset + len(temp),1)
            ThreeColumns.iloc[units, 0] = np.array(temp.luminance)
            ThreeColumns.iloc[units, 1] = np.array(temp.contrast)
            ThreeColumns.iloc[units, 2] = np.array(temp.complexity)
            arr = np.concatenate((arr, units))
            
            # add rating period visual property
            rating_onset = int(float(dm_info[dm_info[1]==r][10][c]))
            rating_offset = int(float(dm_info[dm_info[1]==r][10][c]) + 12)
            
            rating_units = np.arange(rating_onset, rating_offset,1)
            
            expect_units = np.arange(int(float(dm_info[dm_info[1]==r][5][c])), int(float(dm_info[dm_info[1]==r][6][c])),1)
            rating_units = np.concatenate((rating_units, expect_units))
            ThreeColumns.iloc[rating_units, 0] = other_visual_info.loc[0].luminance
            ThreeColumns.iloc[rating_units, 1] = other_visual_info.loc[0].contrast
            ThreeColumns.iloc[rating_units, 2] = other_visual_info.loc[0].complexity
            arr = np.concatenate((arr, rating_units))
            
            
        # fill the rest of the time points with fixation visual property
        fixation_units = np.setdiff1d(np.arange(0,552), arr)
        ThreeColumns.iloc[fixation_units, 0] = other_visual_info.loc[1].luminance
        ThreeColumns.iloc[fixation_units, 1] = other_visual_info.loc[1].contrast
        ThreeColumns.iloc[fixation_units, 2] = other_visual_info.loc[1].complexity
        
        ThreeColumns.to_csv(design_dir + f'subject_{s}_run_{r}.csv', header = True, index = False)

04
0th video
1th video
2th video
3th video
4th video
5th video
6th video
7th video
8th video
9th video
10th video
11th video
12th video
13th video
14th video
15th video
16th video
17th video
18th video
19th video
20th video
21th video
22th video
23th video
24th video
25th video
26th video
27th video
28th video
29th video
30th video
31th video
32th video
33th video
34th video
35th video
05
0th video
1th video
2th video
3th video
4th video
5th video
6th video
7th video
8th video
9th video
10th video
11th video
12th video
13th video
14th video
15th video
16th video
17th video
18th video
19th video
20th video
21th video
22th video
23th video
24th video
25th video
26th video
27th video
28th video
29th video
30th video
31th video
32th video
33th video
34th video
35th video
06
0th video
1th video
2th video
3th video
4th video
5th video
6th video
7th video
8th video
9th video
10th video
11th video
12th video
13th video
14th video
15th video
16th video
17th video
18th video
19th video
20th vide